# Step 2: ANALYZE

In [1]:
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import re
import pandas as pd
import numpy as np

In [12]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as SP_en
from spacy.lang.es.stop_words import STOP_WORDS as SP_es

#!python -m spacy download en
#nlp = spacy.load('en')

#!python -m spacy download es_core_news_md
#nlp_es = spacy.load('es')

In [13]:
nlp_es = spacy.load('es')

OSError: [E050] Can't find model 'es'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [10]:
#import nltk 
#from nltk.corpus import stopwords 
#set(stopwords.words('english'))


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\x385645/nltk_data'
    - 'C:\\Users\\x385645\\AppData\\Local\\Continuum\\anaconda3\\envs\\final_project\\nltk_data'
    - 'C:\\Users\\x385645\\AppData\\Local\\Continuum\\anaconda3\\envs\\final_project\\share\\nltk_data'
    - 'C:\\Users\\x385645\\AppData\\Local\\Continuum\\anaconda3\\envs\\final_project\\lib\\nltk_data'
    - 'C:\\Users\\x385645\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
dic_key_words = \
{   'data analyst'  : ['data analyst','analisis de datos', 
                       'data analytics', 'analitica de datos',
                       'business analyst', 'analisis de negocios',
                       'sql',
                       'python',
                       'business intelligence', 'inteligencia de negocios' , ' bi ' , 
                       'excel', 'microsoft office', 'ms office' ,
                       'dashboard','kpi', 
                       'reporting', 'informes',
                       'tableau', 'powerbi', 'power bi', 'microsoft bi', 'qlik', 'spotfire',
                      ],
 
    'data scientist': ['data scientist', 'cientifico de datos',
                       'data science', 'ciencia de datos',
                       'sql', 
                       'python', ' r ', 
                       ' ml ', 'machine learning','deep learning','neuronal networks', 
                       'pandas', 'numpy', 'scipy', 'web scrapping' , 
                       'data visualization', 'visualizacion de datos','matplotlib', 'seaborn', 
                       'spss',  'statistics',
                       'sklearn','sk learn', 'sk-learn','scikit learn', 'scikit-learn' ,'tensorflow', 'keras',
                       'data wrangling', 'limpieza de datos',
                       ' nlp ', 'a/b testing','a b testing'
                      ],
 
    'data engineer':  ['data engineer', 'ingeniero de datos',
                       'sql', 'nosql', 'no sql', 'bases de datos no relacionales',
                       'python', ' r ', 'scala', 'java ', 'spark', 'hadoop', 'haddop' 'hive', 'impala', 'kafka', 
                       'data wrangling', 'limpieza de datos',
                       'etl','extract, transform and load','map reduce','mapreduce', 
                       'data acquisition', 'adquisicion de datos',
                       'big data', 'bigdata', 'bigquery',
                       'data warehouse', 
                       ' aws ', 'azure',
                       'software engineering'
                      ]
}

## Getting csv

#### > PROFILE:

In [ ]:
df_profile = pd.read_csv(f'..\\data\\raw\\df_profile.csv',
                         sep = ';',
                         encoding= 'utf8'
              )
df_profile

In [ ]:
## ONLY FOR JUPYTER !!!!!!!!!!
df_profile.rename(columns={'Unnamed: 0':'type'},inplace=True)
df_profile.set_index('type', inplace=True)

In [ ]:
df_profile

#### > JOBS:

In [ ]:
df_jobs_raw = pd.read_csv(f'..\\data\\raw\\df_jobs_Madrid_2020-04-18 23.50.csv',
              sep = ';',
              encoding= 'utf8',
              )

In [ ]:
df_jobs = df_jobs_raw

In [ ]:
df_jobs.shape

In [ ]:
df_jobs.head(3)

In [ ]:
# ONLY IN CASE TO CORRECT WRONG DF
#df_jobs.drop(columns='Job htmlJob name', inplace=True)

#df_jobs = df_jobs[['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
#       'Job html', 'Job name', 'Company name', 'Company location',
#       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
#       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
#       'Job info']]

In [ ]:
df_jobs[df_jobs['JOB TITLE']=='data analyst'].shape

In [ ]:
df_jobs[df_jobs['JOB TITLE']=='data scientist'].shape

In [ ]:
df_jobs[df_jobs['JOB TITLE']=='data engineer'].shape

### > Cleaning jobs:

In [ ]:
df_jobs.loc[0,'JOB TITLE':'Current Job Id']

##### * JOB TITLE_ID

In [ ]:
df_jobs['JOB TITLE_ID'] = df_jobs.apply(lambda x: x['JOB TITLE'] +"_"+ str(x['Current Job Id']), axis = 1 )

In [ ]:
df_jobs.shape[0] 

In [ ]:
len(df_jobs['JOB TITLE_ID'].unique())

In [ ]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['JOB TITLE_ID'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

In [ ]:
df_jobs[df_jobs['Current Job Id']==1774326710]

In [ ]:
df_jobs.loc[572,'Job info'] == df_jobs.loc[597,'Job info']

In [ ]:
df_jobs.shape[0] - len(df_jobs['JOB TITLE_ID'].unique())

##### * Current Job Id

In [ ]:
len(df_jobs['Current Job Id'].unique())

In [ ]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['Current Job Id'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

In [ ]:
df_jobs[df_jobs['Current Job Id']==1645918674]

In [ ]:
df_jobs[df_jobs['Current Job Id']==1747233587]

In [ ]:
df_jobs_unique = df_jobs.drop_duplicates(subset=['JOB TITLE_ID'])
df_jobs.shape

### > Language of each job description:

In [ ]:
df_jobs['Job name'].head(10)

In [ ]:
s = df_jobs.loc[6,'Job info']
s

In [ ]:
detect(s)

In [ ]:
df_jobs['Info Language'] = df_jobs['Job info'].apply(lambda x: detect(x))

In [ ]:
df_jobs['Info Language'].unique()

In [ ]:
df_jobs['Seniority Level'].unique()

### > Coincidences with definitions:

In [ ]:
def coincidencias(job_skills, post_skills):
    c = 0
    for i in job_skills: 
        if re.search(i,post_skills): 
            c+=1
    return c
        

In [ ]:
def list_coincidencias(job_skills, post_skills):
    l=[]
    for i in job_skills:
        if re.search(i,post_skills):
            l.append(i)
    return l
        

In [ ]:
def clean_text(text):
    text = text.replace('\n',' ')
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.replace('.',' ')
    text = text.lower()
    
    return text
    

#### trabajos >

In [ ]:
DA_list = dic_key_words.get('data analyst')
DS_list = dic_key_words.get('data scientist')
DE_list = dic_key_words.get('data engineer')

DA_list

In [ ]:
df_jobs['DA coincidences'] = \
df_jobs['Job info'].apply(lambda x: coincidencias(DA_list,clean_text(x)))
df_jobs['DA coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(DA_list,clean_text(x)))

df_jobs['DS coincidences'] = \
df_jobs['Job info'].apply(lambda x: coincidencias(DS_list,clean_text(x)))
df_jobs['DS coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(DS_list,clean_text(x)))

df_jobs['DE coincidences'] = \
df_jobs['Job info'].apply(lambda x: coincidencias(DE_list,clean_text(x)))
df_jobs['DE coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(DE_list,clean_text(x)))

#### perfil >

In [ ]:
df_profile.loc['skills', 'info']

In [ ]:
#### THIS IS ONLY FOR JUPYTER !!!!!!!!!!!!!!!!!!!!
profile_skills = clean_text(df_profile.loc['skills', 'info']).replace('[','').replace(']','').replace("'","").split(', ')
profile_skills

In [ ]:
df_jobs['PROFILE coincidences'] = \
df_jobs['Job info'].apply(lambda x:      coincidencias(profile_skills,clean_text(x)))
df_jobs['PROFILE coincidences_list'] = \
df_jobs['Job info'].apply(lambda x: list_coincidencias(profile_skills,clean_text(x)))

In [ ]:
check = 10

In [ ]:
df_jobs.loc[check ,'Job name']

In [ ]:
dict(df_jobs.loc[check ,'DA coincidences':'PROFILE coincidences_list'])

In [ ]:
clean_text(df_jobs.loc[check ,'Job info'])

In [ ]:
df_jobs.sort_values(by=['PROFILE coincidences'], ascending=False).head(5)